In [ ]:
# %%
# ปรับปรุงโมเดลทำนายอายุเพื่อแก้ปัญหา Overfitting
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Input, Attention, Add
from tensorflow.keras.applications import MobileNetV2, EfficientNetB2, ResNet50V2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import seaborn as sns
import math

# สร้างโฟลเดอร์สำหรับเก็บผลลัพธ์ - เปลี่ยนเป็น model82
OUTPUT_FOLDER = 'model82'
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# กำหนดค่าคงที่ - ปรับเพียงเล็กน้อยจากโมเดลเดิม
IMG_SIZE = 144  # ปรับขึ้นเพียงเล็กน้อยจาก 128 เดิม
BATCH_SIZE = 32  # คงเดิมตามโมเดลเก่า
EPOCHS = 40  # คงเดิมตามโมเดลเก่า
NUM_CLASSES_AGE = 8
RANDOM_STATE = 42

# ตัวแปรสำหรับ path ของข้อมูล
data_parent = 'C:/Users/focus/copy_age_gender_mo2/all_path'  # ปรับตาม path ของคุณ

# อ่านข้อมูลจากทุก fold
data = pd.read_csv(os.path.join(data_parent, 'fold_0_data.txt'), sep='\t')
data1 = pd.read_csv(os.path.join(data_parent, 'fold_1_data.txt'), sep='\t')
data2 = pd.read_csv(os.path.join(data_parent, 'fold_2_data.txt'), sep='\t')
data3 = pd.read_csv(os.path.join(data_parent, 'fold_3_data.txt'), sep='\t')
data4 = pd.read_csv(os.path.join(data_parent, 'fold_4_data.txt'), sep='\t')

# รวมข้อมูลทั้งหมด
total_data = pd.concat([data, data1, data2, data3, data4], ignore_index=True)

# สร้าง path ของรูปภาพ
img_path = []
for row in total_data.iterrows():
    path = os.path.join(data_parent, "faces", row[1].user_id, 
                    f"coarse_tilt_aligned_face.{str(row[1].face_id)}.{row[1].original_image}")
    img_path.append(path)

# สร้าง DataFrame สำหรับการเทรนโมเดล
df = total_data[['age', 'gender', 'x', 'y', 'dx', 'dy']].copy()
df['img_path'] = img_path

# การแบ่งช่วงอายุ
age_mapping = [('(0, 2)', '0-2'), ('2', '0-2'), ('3', '0-2'), ('(4, 6)', '4-6'), 
               ('(8, 12)', '8-13'), ('13', '8-13'), ('22', '15-20'), ('(8, 23)','15-20'), 
               ('23', '25-32'), ('(15, 20)', '15-20'), ('(25, 32)', '25-32'), 
               ('(27, 32)', '25-32'), ('32', '25-32'), ('34', '25-32'), ('29', '25-32'), 
               ('(38, 42)', '38-43'), ('35', '38-43'), ('36', '38-43'), ('42', '48-53'), 
               ('45', '38-43'), ('(38, 43)', '38-43'), ('(38, 42)', '38-43'), 
               ('(38, 48)', '48-53'), ('46', '48-53'), ('(48, 53)', '48-53'), 
               ('55', '48-53'), ('56', '48-53'), ('(60, 100)', '60+'), 
               ('57', '60+'), ('58', '60+')]

age_mapping_dict = {each[0]: each[1] for each in age_mapping}

# คัดกรองข้อมูลที่มีปัญหา
drop_indices = [idx for idx, val in enumerate(df.age) 
                if val == 'None' or pd.isna(val) or val not in age_mapping_dict]

# แปลงค่าอายุ
for idx, each in enumerate(df.age):
    if idx not in drop_indices and each in age_mapping_dict:
        df.loc[idx, 'age'] = age_mapping_dict[each]

# ลบแถวที่มีปัญหา
df = df.drop(labels=drop_indices, axis=0)

# ลบค่า NaN และเพศที่ไม่ทราบ
df = df.dropna()
unbiased_data = df[df.gender != 'u'].copy()

# บันทึกข้อมูลสำหรับอ้างอิง
unbiased_data.to_csv(os.path.join(OUTPUT_FOLDER, 'clean_data.csv'), index=False)

# การแมป label
age_to_label_map = {
    '0-2': 0,
    '4-6': 1,
    '8-13': 2,
    '15-20': 3,
    '25-32': 4,
    '38-43': 5,
    '48-53': 6,
    '60+': 7
}

# แมปย้อนกลับสำหรับการแปลผลลัพธ์
label_to_age_map = {value: key for key, value in age_to_label_map.items()}

# แปลงเป็น numerical labels
unbiased_data['age_label'] = unbiased_data['age'].apply(lambda age: age_to_label_map[age])

# ฟังก์ชั่นสำหรับวิเคราะห์ความไม่สมดุลของข้อมูล
def analyze_age_data_imbalance():
    print("Age Data Distribution Analysis:")
    age_counts = unbiased_data['age'].value_counts()
    print(age_counts)
    
    plt.figure(figsize=(12, 6))
    sns.countplot(data=unbiased_data, x='age', order=age_counts.index)
    plt.title('Age Distribution')
    plt.xlabel('Age Range')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_FOLDER, 'age_distribution.png'))
    
    return age_counts

# ปรับปรุงฟังก์ชั่นโหลดและประมวลผลภาพที่มีการ augmentation ที่หลากหลายมากขึ้น
def load_and_preprocess_image_for_age(img_path, augment=False):
    try:
        img = cv2.imread(img_path)
        if img is None:
            return np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        
        # ปรับปรุงภาพด้วย histogram equalization แยกตามช่องสี
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l_channel)
        merged = cv2.merge((cl, a_channel, b_channel))
        img = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
        
        # ปรับขนาดภาพ
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # การเพิ่มคุณภาพภาพ (Data Augmentation)
        if augment:
            # หมุนภาพ - เพิ่มช่วงการหมุนให้มากขึ้น
            if np.random.random() > 0.5:
                angle = np.random.uniform(-20, 20)  # เพิ่มช่วงการหมุน
                height, width = img.shape[:2]
                matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
                img = cv2.warpAffine(img, matrix, (width, height))
            
            # ปรับความสว่างและคอนทราสต์ - เพิ่มช่วงให้มากขึ้น
            if np.random.random() > 0.4:  # เพิ่มโอกาสการเกิด
                alpha = np.random.uniform(0.7, 1.3)  # เพิ่มช่วงคอนทราสต์
                beta = np.random.uniform(-15, 15)    # เพิ่มช่วงความสว่าง
                img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
            
            # สลับภาพแนวนอน
            if np.random.random() > 0.5:
                img = cv2.flip(img, 1)
            
            # การเลื่อนภาพ - เพิ่มช่วงการเลื่อน
            if np.random.random() > 0.4:
                tx = np.random.uniform(-15, 15)  # เพิ่มช่วงการเลื่อน
                ty = np.random.uniform(-15, 15)
                translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
                img = cv2.warpAffine(img, translation_matrix, (IMG_SIZE, IMG_SIZE))
            
            # เพิ่มสัญญาณรบกวน (noise)
            if np.random.random() > 0.6:  # เพิ่มโอกาสการเกิด
                noise_type = np.random.choice(['gaussian', 'salt_pepper'])
                if noise_type == 'gaussian':
                    noise = np.random.normal(0, 5, img.shape).astype(np.uint8)  # เพิ่มความเข้มของ noise
                    img = cv2.add(img, noise)
                else:
                    # Salt and pepper noise
                    s_vs_p = 0.5
                    amount = 0.01
                    # Salt
                    num_salt = np.ceil(amount * img.size * s_vs_p)
                    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in img.shape]
                    img[coords[0], coords[1], :] = 255
                    # Pepper
                    num_pepper = np.ceil(amount * img.size * (1. - s_vs_p))
                    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in img.shape]
                    img[coords[0], coords[1], :] = 0
                
            # เทคนิค Cutout ที่ปรับปรุง
            if np.random.random() > 0.7:
                num_cutouts = np.random.randint(1, 3)  # อาจมีมากกว่า 1 cutout
                for _ in range(num_cutouts):
                    h, w = img.shape[:2]
                    mask_size_h = int(h * np.random.uniform(0.05, 0.2))  # เพิ่มขนาดสูงสุด
                    mask_size_w = int(w * np.random.uniform(0.05, 0.2))
                    
                    # สุ่มตำแหน่งที่จะตัด
                    top = np.random.randint(0, h - mask_size_h)
                    left = np.random.randint(0, w - mask_size_w)
                    
                    # สร้างพื้นที่สีดำหรือสีเฉลี่ย
                    if np.random.random() > 0.5:
                        img[top:top+mask_size_h, left:left+mask_size_w, :] = 0
                    else:
                        # ใช้สีเฉลี่ยแทน
                        img[top:top+mask_size_h, left:left+mask_size_w, :] = img.mean(axis=(0,1))
                
            # เพิ่ม MixUp (แบบง่าย) สำหรับช่วงอายุที่มีตัวอย่างน้อย (48-53 และ 60+)
            # จำเป็นต้องทำในระดับ generator ไม่ใช่ในฟังก์ชั่นนี้ แต่ระบุเป็นความเห็น
                
        return img
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

# สร้าง Focal Loss เพื่อให้ความสำคัญกับคลาสที่ทำนายยาก
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-9
        y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_sum(loss, axis=-1)
    return focal_loss_fixed

# ฟังก์ชั่น Learning Rate Scheduler ด้วย Cosine Annealing
def cosine_annealing_scheduler(epoch, total_epochs=EPOCHS, initial_lr=0.0001, min_lr=1e-6):
    return min_lr + (initial_lr - min_lr) * (1 + math.cos(math.pi * epoch / total_epochs)) / 2

# ฟังก์ชั่นสร้าง Data Generator ที่สมดุลสำหรับข้อมูลอายุ
def balanced_age_data_generator(img_paths, labels, batch_size=24, augment=True):
    num_samples = len(img_paths)
    
    # คำนวณน้ำหนักสำหรับแต่ละคลาส
    y_integers = labels.values
    class_weights_array = class_weight.compute_class_weight(
        'balanced', 
        classes=np.unique(y_integers), 
        y=y_integers
    )
    class_weights = dict(enumerate(class_weights_array))
    
    # สร้าง indices สำหรับแต่ละคลาส
    class_indices = {}
    for cls in np.unique(y_integers):
        class_indices[cls] = np.where(y_integers == cls)[0]
    
    while True:
        batch_images = []
        batch_labels = []
        
        # สุ่มเลือกข้อมูลแบบสมดุล
        samples_per_class = batch_size // len(class_indices)
        remainder = batch_size % len(class_indices)
        
        for cls, indices in class_indices.items():
            # เพิ่มจำนวนตัวอย่างสำหรับคลาสที่ยากต่อการทำนาย (6, 7)
            boost_factor = 1.0
            if cls in [6, 7]:  # คลาส 48-53 และ 60+
                boost_factor = 1.5
                
            n_samples = int(samples_per_class * boost_factor) + (1 if remainder > 0 else 0)
            remainder -= 1 if remainder > 0 else 0
            
            if len(indices) < n_samples:
                # ถ้ามีตัวอย่างไม่พอ ให้สุ่มซ้ำ
                sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
            else:
                sampled_indices = np.random.choice(indices, size=n_samples, replace=False)
            
            for idx in sampled_indices:
                img = load_and_preprocess_image_for_age(img_paths.iloc[idx], augment=augment)
                batch_images.append(img)
                batch_labels.append(labels.iloc[idx])
        
        # แปลงเป็น numpy arrays
        batch_images = np.array(batch_images) / 255.0
        
        # แปลงเป็น one-hot encoding
        batch_labels = to_categorical(batch_labels, NUM_CLASSES_AGE)
        
        yield batch_images, batch_labels

# ปรับปรุงโมเดลทำนายอายุใหม่
def create_improved_age_model():
    # กลับไปใช้ ResNet50V2 เหมือนเดิม แต่ปรับการโหลด weights และ input size
    base_model = ResNet50V2(
        weights='imagenet', 
        include_top=False, 
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # ล็อคชั้นฟีเจอร์ส่วนแรกๆ เพื่อรักษาฟีเจอร์พื้นฐาน
    # ล็อคน้อยลงจากเดิมเพื่อให้โมเดลเรียนรู้ได้มากขึ้น
    freeze_layers = 50  # ล็อคเพียง 50 เลเยอร์แรกเท่านั้น
    
    for layer in base_model.layers[:freeze_layers]:
        layer.trainable = False
    
    # สร้างโมเดล
    input_img = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    # รับฟีเจอร์จาก base model
    x = base_model(input_img)
    
    # Global Average Pooling ช่วยลดจำนวนพารามิเตอร์และลด overfitting
    x = GlobalAveragePooling2D()(x)
    
    # เพิ่ม regularization และ normalization
    x = BatchNormalization()(x)
    
    # ลดความซับซ้อนของเน็ตเวิร์คลง แต่ไม่ให้มาก regularization จนเกินไป
    x = Dense(
        128, 
        activation='relu', 
        kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),  # ใช้ regularization ที่ไม่มากเกินไป
        activity_regularizer=l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = Dropout(0.4)(x)  # ใช้ dropout rate ที่ไม่มากเกินไป
    
    x = BatchNormalization()(x)
    x = Dense(
        64, 
        activation='relu', 
        kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = Dropout(0.4)(x)
    
    # Output layer - ไม่เปลี่ยนแปลง
    predictions = Dense(NUM_CLASSES_AGE, activation='softmax')(x)
    
    model = Model(inputs=input_img, outputs=predictions)
    
    # ใช้ learning rate เริ่มต้นที่มากขึ้นเพื่อเรียนรู้ได้เร็วขึ้น
    optimizer = Adam(learning_rate=0.0002)  # เพิ่มจาก 0.0001 เดิม
    
    # ยังคงใช้ categorical_crossentropy เพื่อให้สอดคล้องกับโมเดลเดิม
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# ฟังก์ชั่นเทรนโมเดลอายุที่ปรับปรุงแล้ว
def train_improved_age_model():
    # แบ่งข้อมูลสำหรับการเทรนและทดสอบแบบมีการ stratify
    X_train, X_test, y_train, y_test = train_test_split(
        unbiased_data['img_path'],
        unbiased_data['age_label'],
        test_size=0.2,
        random_state=RANDOM_STATE,
        stratify=unbiased_data['age_label']
    )
    
    # แสดงข้อมูลการแบ่ง
    print(f"Training data: {len(X_train)} samples")
    print(f"Testing data: {len(X_test)} samples")
    
    # คำนวณ class weights สำหรับข้อมูลที่ไม่สมดุล
    # เพิ่มน้ำหนักสำหรับคลาสที่มีตัวอย่างน้อย
    class_weights = class_weight.compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    
    # เพิ่มน้ำหนักให้กับคลาสที่ยากต่อการทำนาย (48-53, 60+)
    for i in [6, 7]:  # คลาส 48-53 และ 60+
        if i in range(len(class_weights)):
            class_weights[i] *= 1.5  # เพิ่มน้ำหนัก 50%
    
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
    
    # สร้างโมเดล
    model = create_improved_age_model()
    
    # สร้าง callbacks
    # บันทึกโมเดลที่ดีที่สุดโดยพิจารณาจาก val_loss
    checkpoint = ModelCheckpoint(
        os.path.join(OUTPUT_FOLDER, 'age_model_best.h5'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
        mode='min'
    )
    
    # ปรับ early stopping - ใช้ค่าความอดทนที่เหมาะสม
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=1,
        restore_best_weights=True,
        mode='min'
    )
    
    # ใช้ ReduceLROnPlateau แทน cosine annealing เพื่อให้ปรับ LR ตามการพัฒนาของโมเดล
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,  # ลด learning rate มากขึ้นเมื่อไม่มีการปรับปรุง
        patience=5,
        min_lr=1e-6,
        verbose=1,
        mode='min'
    )
    
    # สร้าง generators
    train_generator = balanced_age_data_generator(X_train, y_train, BATCH_SIZE, augment=True)
    validation_generator = balanced_age_data_generator(X_test, y_test, BATCH_SIZE, augment=False)
    
    # เทรนโมเดล
    steps_per_epoch = len(X_train) // BATCH_SIZE
    validation_steps = len(X_test) // BATCH_SIZE
    
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[checkpoint, early_stopping, reduce_lr],
        class_weight=class_weight_dict,
        validation_freq=1
    )
    
    # บันทึกโมเดลสุดท้าย
    model.save(os.path.join(OUTPUT_FOLDER, 'age_model_final.h5'))
    
    # สร้างกราฟแสดงผลการเทรน
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Age Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Age Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_FOLDER, 'age_model_training.png'))
    
    # ประเมินโมเดลกับข้อมูลทดสอบ
    print("\nAge Model Evaluation:")
    
    # โหลดโมเดลที่ดีที่สุด
    model = load_model(os.path.join(OUTPUT_FOLDER, 'age_model_best.h5'))
    
    # สร้าง predictions สำหรับข้อมูลทดสอบ - ปรับปรุงวิธีการประเมิน
    predictions = []
    true_labels = []
    class_probabilities = {}  # เก็บความน่าจะเป็นของแต่ละคลาส
    
    # สร้าง dict สำหรับเก็บความน่าจะเป็น
    for cls in range(NUM_CLASSES_AGE):
        class_probabilities[cls] = []
    
    # สุ่มเลือกตัวอย่างเพื่อประหยัดเวลาในการประเมิน
    test_indices = np.random.choice(len(X_test), min(1000, len(X_test)), replace=False)
    
    for idx in test_indices:
        img_path = X_test.iloc[idx]
        true_label = y_test.iloc[idx]
        
        img = load_and_preprocess_image_for_age(img_path)
        img = np.expand_dims(img, axis=0) / 255.0
        
        pred = model.predict(img, verbose=0)
        pred_label = np.argmax(pred[0])
        
        # เก็บผลลัพธ์
        predictions.append(pred_label)
        true_labels.append(true_label)
        
        # เก็บความน่าจะเป็นของแต่ละคลาส
        for cls in range(NUM_CLASSES_AGE):
            class_probabilities[cls].append(pred[0][cls])
    
    # สร้างรายงานการประเมิน
    age_categories = list(age_to_label_map.keys())
    print(classification_report(true_labels, predictions, 
                               target_names=age_categories))
    
    # สร้าง confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=age_categories,
               yticklabels=age_categories)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Age Prediction Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_FOLDER, 'age_confusion_matrix.png'))
    
    # สร้างกราฟแสดงความน่าจะเป็นเฉลี่ยของแต่ละคลาส
    plt.figure(figsize=(12, 6))
    avg_probabilities = [np.mean(class_probabilities[cls]) for cls in range(NUM_CLASSES_AGE)]
    sns.barplot(x=age_categories, y=avg_probabilities)
    plt.title('Average Prediction Confidence by Age Group')
    plt.xlabel('Age Group')
    plt.ylabel('Average Confidence')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_FOLDER, 'age_prediction_confidence.png'))
    
    return model, history

# ฟังก์ชั่นทำนายอายุจากภาพ
def predict_age(image_path, model):
    """
    ทำนายอายุจากภาพใบหน้า
    """
    # อ่านภาพ
    img = cv2.imread(image_path)
    if img is None:
        return "Cannot read image"
    
    # ตรวจจับใบหน้า
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # สร้างภาพผลลัพธ์
    result_img = img.copy()
    
    # ประมวลผลแต่ละใบหน้าที่ตรวจพบ
    for (x, y, w, h) in faces:
        x1, y1, x2, y2 = x, y, x+w, y+h
        
        # ตัดภาพใบหน้า
        face_img = img[y1:y2, x1:x2]
        
        # ตรวจสอบว่าใบหน้าถูกต้อง
        if face_img.size == 0 or face_img.shape[0] == 0 or face_img.shape[1] == 0:
            continue
        
        # ปรับปรุงภาพใบหน้า
        lab = cv2.cvtColor(face_img, cv2.COLOR_BGR2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l_channel)
        merged = cv2.merge((cl, a_channel, b_channel))
        enhanced_face = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
        
        # ปรับขนาดสำหรับการทำนาย
        face_resized = cv2.resize(enhanced_face, (IMG_SIZE, IMG_SIZE))
        face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
        face_rgb = face_rgb / 255.0
        face_rgb = np.expand_dims(face_rgb, axis=0)
        
        # ทำนายอายุ
        age_pred = model.predict(face_rgb, verbose=0)
        age_class = np.argmax(age_pred[0])
        age_prob = np.max(age_pred[0]) * 100
        
        age_label = label_to_age_map[age_class]
        
        # วาดกรอบ
        cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # เพิ่มผลลัพธ์ลงในภาพ
        label_text = f"Age: {age_label} ({age_prob:.1f}%)"
        cv2.putText(result_img, label_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # บันทึกภาพผลลัพธ์
    result_path = os.path.join(OUTPUT_FOLDER, 'result_' + os.path.basename(image_path))
    cv2.imwrite(result_path, result_img)
    
    return result_path

# ฟังก์ชั่นทดสอบด้วยภาพ
def test_with_image(image_path, model=None):
    """
    ทดสอบโมเดลกับภาพและแสดงผลลัพธ์
    """
    # อ่านภาพ
    img = cv2.imread(image_path)
    if img is None:
        print("Cannot read image")
        return
    
    # ตรวจจับใบหน้า
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # สร้างภาพผลลัพธ์
    result_img = img.copy()
    
    # โหลดโมเดล (ถ้าไม่ได้ระบุ)
    if model is None:
        model = load_model(os.path.join(OUTPUT_FOLDER, 'age_model_best.h5'))
    
    # ประมวลผลแต่ละใบหน้าที่ตรวจพบ
    for (x, y, w, h) in faces:
        # ตัดภาพใบหน้า
        face_img = img[y:y+h, x:x+w]
        
        # ปรับปรุงภาพใบหน้า
        lab = cv2.cvtColor(face_img, cv2.COLOR_BGR2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l_channel)
        merged = cv2.merge((cl, a_channel, b_channel))
        enhanced_face = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
        
        # ปรับขนาด
        face_resized = cv2.resize(enhanced_face, (IMG_SIZE, IMG_SIZE))
        face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
        face_rgb = face_rgb / 255.0
        face_rgb = np.expand_dims(face_rgb, axis=0)
        
        # ทำนาย
        age_pred = model.predict(face_rgb, verbose=0)
        age_class = np.argmax(age_pred[0])
        age_prob = np.max(age_pred[0]) * 100
        
        age_label = label_to_age_map[age_class]
        
        # วาดกรอบและป้ายกำกับ
        cv2.rectangle(result_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        label_text = f"Age: {age_label} ({age_prob:.1f}%)"
        cv2.putText(result_img, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # บันทึกและแสดงผลลัพธ์
    result_path = os.path.join(OUTPUT_FOLDER, 'result_' + os.path.basename(image_path))
    cv2.imwrite(result_path, result_img)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Age Prediction Result')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_FOLDER, 'prediction_result.png'))
    
    return result_path


if __name__ == "__main__":
    print("=" * 50)
    print("Improved Age Prediction from Face (Anti-Overfitting) - model82")
    print("=" * 50)
    
    # 1. วิเคราะห์ข้อมูล
    print("\n1. Analyzing age data...")
    age_counts = analyze_age_data_imbalance()
    print("\nData Summary:")
    print(f"Total samples: {len(unbiased_data)}")
    print(f"Number of age classes: {NUM_CLASSES_AGE}")
    
    # 2. เทรนโมเดลอายุที่ปรับปรุงแล้ว
    print("\n2. Training improved age model...")
    age_model, age_history = train_improved_age_model()
    
    # 3. ทดสอบกับภาพ
    test_img = "C:/Users/focus/Pictures/USE_to_test_ml/image_th.png"  # แก้ไขเป็น path ของภาพที่ต้องการทดสอบ
    result_path = test_with_image(test_img, age_model)
    
    print(f"\nPrediction completed. Result saved at: {result_path}")